In [4]:
import pickle
import pandas as pd

enron_df = pd.read_pickle('enron_students.pkl')

# # Read dictionary pkl file
# with open('email_to_departments.pkl', 'rb') as fp:
#     email_to_departments = pickle.load(fp)
#     print('email_to_departments dictionary loaded from pkl file')

In [5]:
# data cleaning
enron_df['To'] = enron_df['To'].fillna('')
enron_df['From'] = enron_df['From'].fillna('')
enron_df['X-From'] = enron_df['X-From'].fillna('')
enron_df['X-To'] = enron_df['X-To'].fillna('')
enron_df['X-cc'] = enron_df['X-cc'].fillna('')
enron_df['X-bcc'] = enron_df['X-bcc'].fillna('')


In [44]:
import re
import pandas as pd

def extract_emails(text):
    if pd.isna(text):
        return []
    # Regex to match email addresses
    email_pattern = r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b'
    return re.findall(email_pattern, text)

def extract_single_email_from_text(text):
    emails = extract_emails(text)
    if (len(emails) > 1):
        print(f"Multiple emails found in: {text}")
    elif len(emails) == 1:
        return emails[0]
    print(f"Could not extract email from: {text}")
    return None

def extract_sent_to_emails(text):
    emails = extract_emails(text)
    if len(emails) > 0:
        return emails
    return []

def is_between_ect_and_ees(email_to_depart_dict, row, is_including_cc_bcc=False, additional_check=True):
    sender_email = extract_single_email_from_text(row['From'])
    if sender_email in email_to_depart_dict:
        sender_department = email_to_depart_dict[sender_email]
        if sender_department != 'ECT' and sender_department != 'EES':
            return False
        else:
            if is_including_cc_bcc:
                reciever_emails = extract_sent_to_emails(row['To']) + extract_sent_to_emails(row['X-To'])
            else:
                reciever_emails = extract_sent_to_emails(row['To']) + extract_sent_to_emails(row['X-To']) + extract_sent_to_emails(row['X-cc']) + extract_sent_to_emails(row['X-bcc'])
            
            for email in reciever_emails:
                if email in email_to_depart_dict:
                    reciever_department = email_to_depart_dict[email]

                    if sender_department == 'ECT' and reciever_department == 'EES':
                        return True
                    elif sender_department == 'EES' and reciever_department == 'ECT':
                        return True
                    
            if additional_check:
                if 'ECT' in row['X-To']:
                    if 'EES' in row['X-From']:
                        return True
                if 'EES' in row['X-To']:
                    if 'ECT' in row['X-From']:
                        return True
      
            return False
    
    else:
        if additional_check:
            if 'ECT' in row['X-To']:
                if 'EES' in row['X-From']:
                    return True
            if 'EES' in row['X-To']:
                if 'ECT' in row['X-From']:
                    return True
        return False
    


    

In [29]:

def build_dict():
    email_to_departments = {}
    email_to_departments.update({
            'smith.day@enron.com' : 'ECT'
            ,'janet.wallis@enron.com' : 'ECT'
            ,'daren.farmer@enron.com' : 'ECT'
            ,'stuart.zisman@enron.com' : 'ECT'
            ,'melissa.murphy@enron.com' : 'ECT'
            ,'thresa.allen@enron.com' : 'ECT'
            ,'leslie.reeves@enron.com' : 'ECT'
            ,'stacey.white@enron.com' : 'ECT'
            ,'janet.wallis@enron.com' : 'ECT'
            ,'daren.farmer@enron.com' : 'ECT'
            ,'brad.nebergall@enron.com': 'ECT'
            ,'colleen.koenig@enron.com': 'EES'
            ,'john.kinser@enron.com': 'ECT'
            ,'jeff.merola@enron.com': 'EES'
            })
    
    email_to_departments.update({'eric.bass@enron.com': 'ECT'})

    multiple_count = 0
    # TODO: implement check for multiple emails in X-From and From. should be only one email
    for index, row in enron_df.iterrows():
        x_emails = extract_emails(row['X-From'])
        from_emails = set([mail.lower() for mail in (extract_emails(row['From']) + x_emails)])
        to_emails = set([mail.lower() for mail in (extract_emails(row['To']) + extract_emails(row['X-To']))])
        if len(from_emails) == 0:
            continue
        if len(from_emails) > 1:
            from_emails = [mail for mail in from_emails if not mail.startswith('imceanotes')]
            if len(from_emails) > 1 or len(from_emails) == 0:
                multiple_count += 1

        for email in from_emails:
            email = email.lower()
            # TODO: validate company email starts with enron.com
            if "enron.com" not in email and email not in email_to_departments:
                email_to_departments.update({email: 'NA'})
            else:
                if email in email_to_departments and email_to_departments[email] != 'Other':
                    continue
                elif 'EES' in row['X-From']:
                    email_to_departments.update({email: 'EES'})
                    if len(to_emails) == 1:
                        email_to_departments.update({list(to_emails)[0]: 'ECT'})
                elif 'ECT' in row['X-From']:
                    email_to_departments.update({email: 'ECT'})
                    if len(to_emails) == 1:
                        email_to_departments.update({list(to_emails)[0]: 'EES'})
                else:
                    email_to_departments.update({email: 'Other'})

        # new addition - need to check
        if len(to_emails) == 1:
            if 'ECT' in row['X-To']:
                email_to_departments.update({list(to_emails)[0]: 'ECT'})
            elif 'EES' in row['X-To']:
                email_to_departments.update({list(to_emails)[0]: 'EES'})

    email_to_departments.update({
            'smith.day@enron.com' : 'ECT'
            ,'janet.wallis@enron.com' : 'ECT'
            ,'daren.farmer@enron.com' : 'ECT'
            ,'stuart.zisman@enron.com' : 'ECT'
            ,'melissa.murphy@enron.com' : 'ECT'
            ,'thresa.allen@enron.com' : 'ECT'
            ,'leslie.reeves@enron.com' : 'ECT'
            ,'stacey.white@enron.com' : 'ECT'
            ,'janet.wallis@enron.com' : 'ECT'
            ,'daren.farmer@enron.com' : 'ECT'
            ,'brad.nebergall@enron.com': 'ECT'
            ,'colleen.koenig@enron.com': 'EES'
            ,'john.kinser@enron.com': 'ECT'
            ,'jeff.merola@enron.com	': 'EES'
            ,'scott.mills@enron.com': 'ECT'
            ,'marilyn.colbert@enron.com': 'ECT'
            ,'molly.harris@enron.com': 'ECT'
            ,'joseph.wagner@enron.com': 'ECT'
            ,'stuart.zisman@enron.com,' : 'ECT'
            ,'janet.wallis@enron.com' : 'ECT'
            ,'daren.farmer@enron.com' : 'ECT'
            ,'stephanie.gardner@enron.com' : 'EES'

            })
    
    return email_to_departments


email_to_department_dict = build_dict()

In [30]:
len(email_to_department_dict)

20638

In [31]:
enron_df['is_between_ect_and_ees'] = enron_df.apply(lambda row: is_between_ect_and_ees(email_to_department_dict, row), axis=1)

Could not extract email from: u@d.h
Could not extract email from: pep <performance.>


In [41]:
if 'cheryl.dudley@enron.com' in email_to_department_dict:
    print('yes')

yes


In [83]:
# remove all spaces 
enron_df['violated_rules'] = enron_df['violated_rules'].apply(lambda x: x.replace(' ', ''))

# apply split ',' on violated rules
enron_df['violated_rules'] = enron_df['violated_rules'].apply(lambda x: x.split(',')) 

In [39]:
pd.set_option('display.max_colwidth', 1000)
enron_df_not_catched_1_1 = enron_df[(enron_df['is_between_ect_and_ees'] == False) & (enron_df['violated_rules'].apply(lambda x: '1.1' in x ) & (enron_df['X-To'].str.contains('ECT')))]
enron_df_not_catched_1_1.head(10)
# len(enron_df_not_catched_1_1)

# print(is_between_ect_and_ees(test, enron_df_not_catched_1_1.iloc[0], False))


,Date,From,To,X-To,X-From,X-cc,X-bcc,Subject,email_body,verdict,violated_rules,is_between_ect_and_ees
109452,"Fri, 16 Mar 2001 11:58:00 -0800 (PST)",cheryl.dudley@enron.com,elsa.villarreal@enron.com,Elsa Villarreal <Elsa Villarreal/HOU/ECT@ECT>,Cheryl Dudley <Cheryl Dudley/HOU/ECT@ECT>,"Amelia Alland <Amelia Alland/HOU/ECT@ECT>, Daren J Farmer <Daren J Farmer/HOU/ECT@ECT>, Kimberlee A Bennick <Kimberlee A Bennick/HOU/ECT@ECT>",,"Re: Exxon Company, USA Global # 96035668/Sitara # 212225","I have re-checked Sitara & verified that the above deal was a Term, Firm deal. This resulted in a GTC Purchase Firm contract being set-up in the Global Contracts System. The term of the GTC was set as the term of the deal (4/1/00 thru 3/31/01) with no evergreen, meaning that the contract will expire at the end of this month.\n\nTo the best of my knowledge, all GTC Purchase Firms have the same language & they do not have evergreen. They expire at the end of the primary term.\n\nSo, unless Exxon negotiated a deal with different, special language to allow for evergreen & included something about a ""10 day notification"" to terminate, there isn't any reason to ""terminate"" this contract.\n\nIf this is the case, please advise. Otherwise, I don't think we need to do anything. The contract will expire at the end of this month on its own.\n\n\n\n\n\n\nElsa Villarreal\n03/15/2001 02:42 PM\nTo:\tAmelia Alland/HOU/ECT@ECT, Cheryl Dudley/HOU/ECT@ECT\ncc:\tDaren J Farmer/HOU/ECT@ECT \nSubjec...",BLOCK,1.1,False
112429,"Mon, 30 Apr 2001 12:36:59 -0700 (PDT)",george.weissman@enron.com,robert.cotten@enron.com,"Cotten, Robert </O=ENRON/OU=NA/CN=RECIPIENTS/CN=NOTESADDR/CN=4E8D3E3-CB08A343-862564A6-6A8FC1>, Donald P Reinhardt/HOU/ECT@ENRON <??SDonald P Reinhardt/HOU/ECT@ENRON>","Weissman, George </O=ENRON/OU=NA/CN=RECIPIENTS/CN=NOTESADDR/CN=88372C14-8DEBAA6B-862564B5-68FD6F>","Daren J Farmer/HOU/ECT@ENRON <??SDaren J Farmer/HOU/ECT@ENRON>, Peyton, John </O=ENRON/OU=NA/CN=RECIPIENTS/CN=NOTESADDR/CN=4F6F41D7-6F85666D-862566A6-58EF68>, Melissa Graves/HOU/ECT@ENRON <??SMelissa Graves/HOU/ECT@ENRON>",,"Conoco, Inc. Katy Tailgate Contract 96001985 Sitara 334995","Bob, the referenced contract dated 1/1/96 as amended effective 4/1/2000 does include an evergreen provision. Thus, please add May, 2001 to the existing sitara deal, 334995, at the current price, 90% of IFHSC, for a volume of 155 Mmbtu/d.\n\n Don, please prepare and circulate a termination letter for 96001985.\n\nGeorge x3-6992\n\n\n---------------------- Forwarded by George Weissman/HOU/ECT on 04/30/2001 02:11 PM ---------------------------\n\n\nJohn Peyton\n04/30/2001 01:08 PM\nTo:\tGeorge Weissman/HOU/ECT@ECT\ncc:\t \nSubject:\tConoco Katy Contract\n\nPlease send Conoco's Mike Luchetti two month-to-month contracts for April and May volumes of around 150-200 MMBtu/day. This is for Facility number 10077 - Exxon Plant HPL Katy for gas that was being purchased under a contract that expired April 1, 2001 - contract no 012-19785-113.\n\nThank You,\n\nJohn\n",BLOCK,1.1,False
112430,"Tue, 1 May 2001 11:24:48 -0700 (PDT)",robert.cotten@enron.com,"george.weissman@enron.com, joyce.viltz@enron.com","Weissman, George </O=ENRON/OU=NA/CN=RECIPIENTS/CN=NOTESADDR/CN=88372C14-8DEBAA6B-862564B5-68FD6F>, Melissa Graves/HOU/ECT@ENRON <??SMelissa Graves/HOU/ECT@ENRON>, Daren J Farmer/HOU/ECT@ENRON <??SDaren J Farmer/HOU/ECT@ENRON>, Vance L Taylor/HOU/ECT@ENRON <??SVance L Taylor/HOU/ECT@ENRON>, Susan Smith/HOU/ECT@ENRON <??SSusan Smith/HOU/ECT@ENRON>, Viltz, Joyce </O=ENRON/OU=NA/CN=RECIPIENTS/CN=JVILTZ>","Cotten, Robert </O=ENRON/OU=NA/CN=RECIPIENTS/CN=NOTESADDR/CN=4E8D3E3-CB08A343-862564A6-6A8FC1>",,,"Seacrest Meter # 0435 - April, 2001","The confirmed nomination for Barrett Resources has been zeroed and a like number added to Seneca.\n\nBob\n\n---------------------- Forwarded by Robert Cotten/HOU/ECT on 05/01/2001 01:18 PM ---------------------------\n \n\tEnron North America Corp. From: Robert Cotten 04/26/2001 11:48 AM\t\n

In [115]:
def update_dict(enron_df_not_catched_1_1, email_to_depart):
    # iterate over the rows of the dataframe enron_df_not_catched_1_1
    updated = 0
    for index, row in enron_df_not_catched_1_1.iterrows():
        x_emails = extract_emails(row['X-From'])
        emails = set([mail.lower() for mail in (extract_emails(row['From']) + x_emails)])
        sent_to_emails = extract_sent_to_emails(row['To']) + extract_sent_to_emails(row['X-To'])
        if len(emails) == 0:
            continue
        if len(emails) > 1:
            emails = [mail for mail in emails if not mail.startswith('imceanotes')]
            if len(emails) > 1 or len(emails) == 0:
                # print(f"From: {row['From']} X-From: {row['X-From']} has multiple emails: {emails}")
                continue
        for email in emails:
            email = email.lower()
            # if from email is in dictionary, get the department
            if email in email_to_depart:
                depart = email_to_depart[email]
                if depart == 'ECT':
                    if len(sent_to_emails) == 1:
                        email_to_depart[sent_to_emails[0]] = 'EES'
                        updated += 1
                elif depart == 'EES':
                    if len(sent_to_emails) == 1:
                        email_to_depart[sent_to_emails[0]] = 'ECT'
                        updated += 1

    print(f"Updated {updated} emails")
    


update_dict(enron_df_not_catched_1_1, email_to_department_dict)


Updated 44 emails


* HERE WE START USER TO LOCATION *


In [5]:
def get_email_to_location_dict(df):
    email_to_location = {}
    for index, row in enron_df.iterrows():
        x_emails = extract_emails(row['X-From'])
        from_emails = set([mail.lower() for mail in (extract_emails(row['From']) + x_emails)])
        if '/OU=EU/' in row['X-From']:
            for email in from_emails:
                email_to_location.update({email: 'EU'})
        elif '/HOU/' in row['X-From']:
            for email in from_emails:
                email_to_location.update({email: 'NA'})
        elif '/OU=NA/' in row['X-From']:
            for email in from_emails:
                email_to_location.update({email: 'NA'})

    # iterate over the dataframe to get 
    violated_rule_1_2 = df[df['violated_rules'].apply(lambda x: '1.2' in x)]

    for index, row in violated_rule_1_2.iterrows():
        from_emails = extract_emails(row['X-From'])
        from_emails = list(set([mail.lower() for mail in (extract_emails(row['From']) + from_emails)]))
        if '/OU=EU/' in row['X-From']:
            for email in from_emails:
                email_to_location.update({email: 'EU'})
        elif '/HOU/' in row['X-From']:
            for email in from_emails:
                email_to_location.update({email: 'NA'})
        elif '/OU=NA/' in row['X-From']:
            for email in from_emails:
                email_to_location.update({email: 'NA'})
        
        sent_to_email = extract_emails(row['X-To'])
        if len(sent_to_email) == 1:
            if from_emails[0] not in email_to_location:
                continue
            if email_to_location[from_emails[0]] == 'EU':
                email_to_location.update({sent_to_email[0]: 'NA'})
            elif email_to_location[from_emails[0]] == 'NA':
                email_to_location.update({sent_to_email[0]: 'EU'})
            
    return email_to_location


email_to_location = get_email_to_location_dict(enron_df)

In [26]:
def is_between_EU_and_NA(row, email_to_location):
    from_email = extract_emails(row['From'])
    from_email = extract_emails(row['X-From']) + from_email
    from_email = list(set([mail.lower() for mail in from_email]))
    to_email = extract_emails(row['To'])
    to_email = extract_emails(row['X-To']) + to_email
    to_email = list(set([mail.lower() for mail in to_email]))
    if len(from_email) == 0 or len(to_email) == 0:
        return False
    if from_email[0] in email_to_location:
        if email_to_location[from_email[0]] == 'EU':
            for email in to_email:
                if email in email_to_location and email_to_location[email] == 'NA':
                    return True
        if email_to_location[from_email[0]] == 'NA':
            for email in to_email:
                if email in email_to_location and email_to_location[email] == 'EU':
                    return True
    
    return False

In [27]:
enron_df['is_EU_To_NA'] = enron_df.apply(lambda row: is_between_EU_and_NA(row, email_to_location), axis=1)

In [3]:
not_working = enron_df[(enron_df['is_EU_To_NA'] == False) & (enron_df['violated_rules'].apply(lambda x: '1.2' in x))]
not_working.head(10)

NameError: name 'enron_df' is not defined

In [43]:
print(len(email_to_location))
# enron_df.head(10)
pd.set_option('display.max_colwidth', 50)

3985


In [179]:
def get_c_suit_dict():
    emails = set({
        'kenneth.lay@enron.com',
        'ken.skilling@enron.com',
        'rbowers@nyiso.com',
        'michael.brown@enron.com',
        'john.sherriff@enron.com',
        'jeffrey.sherrick@enron.com',
        'a..shankman@enron.com',
        'ken.rice@enron.com',
        'greg.piper@enron.com',
        'mark.metts@enron.com',
        'coo.jeff@enron.com',
        'rebecca.mcdonald@enron.com',
        'danny.mccarty@enron.com',
        'dan.leff@enron.com',
        'john.lavorato@enron.com',
        'mark.koenig@enron.com',
        'louise.kitchen@enron.com',
        'stanley.horton@enron.com',
        '40enron@enron.com', # for some reason is tagged for stanley horton
        'ben.glisan@enron.com',
        'mark.frevert@enron.com',
        'andrew.fastow@enron.com',
        'jr..legal@enron.com',
        'derrick@enron.com',
        'david.delainey@enron.com',
        'richard.causey@enron.com',
        'michael.brown@enron.com',
        'raymond.bowen@enron.com',
    })

    violated_rule_1_3 = enron_df[enron_df['violated_rules'].apply(lambda x: '2.2' in x)]

    for index, row in violated_rule_1_3.iterrows():
        from_emails = extract_emails(row['X-From'])
        from_emails = list(set([mail.lower() for mail in (extract_emails(row['From']) + from_emails)]))
        to_email = extract_emails(row['To'])
        to_email = extract_emails(row['X-To']) + to_email
        to_email = list(set([mail.lower() for mail in to_email]))

        if len(from_emails) == 0:
            continue
        for email in from_emails:
            emails.add(email)
            # print(f"Email: {email} is in C-Suit")

        # for email in to_email:
        #     emails.add(email)
        #     print(f"Email: {email} is in C-Suit")

    return emails

c_suit_emails = get_c_suit_dict()

In [112]:
def is_in_c_suit(row, c_suit_emails):
    # Extract and clean email addresses from 'From' fields
    from_emails = extract_emails(row['X-From']) + extract_emails(row['From'])
    from_emails = list(set(email.lower() for email in from_emails))

    # Extract and clean email addresses from 'To' fields
    to_email = extract_emails(row['X-To']) + extract_emails(row['To'])
    to_email = list(set(email.lower() for email in to_email))

    # Check if both sender and recipient are in the c-suite email list
    return any(email in c_suit_emails for email in from_emails) and any(email in c_suit_emails for email in to_email)



In [118]:
enron_df['is_c_suit'] = enron_df.apply(lambda row: is_in_c_suit(row, c_suit_emails), axis=1)

In [ ]:
pd.set_option('display.max_colwidth', 100)

In [1]:
def get_from_to_emails(row):
    from_emails = extract_emails(row['X-From']) + extract_emails(row['From'])
    from_emails = list(set(email.lower() for email in from_emails))

    # Extract and clean email addresses from 'To' fields
    to_email = extract_emails(row['X-To']) + extract_emails(row['To'])
    to_email = list(set(email.lower() for email in to_email))

    return from_emails, to_email

In [2]:
def is_leaving_corporate(row):
    from_emails, to_email = get_from_to_emails(row)
    if len(from_emails) == 0 or len(to_email) == 0:
        return False
    
    for email in to_email:
        if 'enron.com' not in email:
            return True

    return False


def is_leaving_ect(row, email_to_depart):
    from_emails, to_email = get_from_to_emails(row)
    if len(from_emails) == 0 or len(to_email) == 0:
        return False

    for email in from_emails:
        if email in email_to_depart and email_to_depart[email] == 'ECT':
            for t_email in to_email:
                if t_email in email_to_depart and email_to_depart[t_email] != 'ECT':
                    return True

    return False

In [42]:
class StaticAnalyzer:
    
    @staticmethod
    def is_violating_rule_1_1(row):
        return is_between_ect_and_ees(email_to_department_dict, row, is_including_cc_bcc=False, additional_check=True)
    
    
    @staticmethod
    def is_violating_rule_1_2(row):
        return is_between_EU_and_NA(row, email_to_location)
    
    
    @staticmethod
    def is_violating_rule_1_3(row):
        return is_leaving_corporate(row)
    
    
    @staticmethod
    def is_violating_rule_2_1(row):
        return is_leaving_ect(row, email_to_department_dict)
    
    
    @staticmethod
    def is_violating_rule_2_2(row):
        return is_in_c_suit(row, c_suit_emails)
    
    @staticmethod
    def is_violating_rule_2_3(row):
        return is_leaving_corporate(row)

In [ ]:
import dataclasses

@dataclasses.dataclass()
class ContentAnalysisResult:
    is_sensitive: bool = False
    quids_counter: int = 0
    has_pii: bool = False
    
@dataclasses.dataclass()
class TopicAnalysisResult:
    is_legal: bool = False
    is_business: bool = False
    is_finance: bool = False
    
@dataclasses.dataclass()
class EnforcerResult:
    violated_rules = []
    
    def is_allowed(self)-> bool:
        return len(self.violated_rules) == 0
    

In [ ]:
class ContentAnalyzer:
    
    def analyze(self, row) -> ContentAnalysisResult:
        pass #todo maor


In [ ]:
import pickle
class TopicAnalyzer:
    # todo efi
    def __init__(self):
        self.model = pickle.load(open("random_forest_model.pkl", 'rb'))
        
    def predict(self, row) -> TopicAnalysisResult:
        result = TopicAnalysisResult()
        result.is_legal = self.is_legal(row)
        result.is_business = self.is_business(row)
        result.is_finance = self.is_finance(row)
        
        return result

    def is_business(self, row):
        
        pass
    
    
    def is_finance(self, row):
        pass
    
    
    def is_legal(self, row):
        pass
    
    

In [1]:
class Enforcer:

    def __init__(self):
        self.static_analyzer = StaticAnalyzer()
        self.topic_analyzer = TopicAnalyzer()
        self.content_analyzer = ContentAnalyzer()
        
    def _pre_process(self, mail_row):
        # todo efi
        pass
    
        
    def enforce(self, mail_row) -> EnforcerResult:
        result = EnforcerResult()
        processed_row = self._pre_process(mail_row)
        
        static_rule_violations_by_rule_id: {str : bool} = self.analyze_static(processed_row)
        content_analysis: ContentAnalysisResult = None
        topic_analysis: TopicAnalysisResult = None
    
        # static analysis
        if any(static_rule_violations_by_rule_id.values()) is False:
            print(f'After static analysis, The Email {mail_row} is not violating any rule and therefore ALLOWED')
            return result
        
        if self.only_this_rule_violated(static_rule_violations_by_rule_id, rule_id='2,3'):
            print(f'Only rule 2.3 is potentially violated in {mail_row}, therefore, we can skip topic analysis')
            content_analysis: ContentAnalysisResult = self.content_analyzer.analyze(processed_row)
            
            if self._is_violating_rule_2_3(content_analysis):
                print(f'Email {mail_row} is violating rule 2.3 due to content analysis: {content_analysis} and therefore BLOCKED')
                result.violated_rules.append('2.3')
            else:
                print(f'Email {mail_row} is not violating any rule due to content analysis: {content_analysis} and therefore ALLOWED')
            return result
        
        print(f'Will analyze topic for {mail_row}, as all rules left required topic analysis')
        topic_analysis = self.topic_analyzer.predict(processed_row)
        
        if self._only_1_rules_are_violated(static_rule_violations_by_rule_id):
            print(f'Only Policy 1# rules are violated statically, can skip content analysis')
        else:
            print(f'Will analyze content for {mail_row}, as all rules left required content analysis (2)')
            content_analysis: ContentAnalysisResult = self.content_analyzer.analyze(processed_row)
        
        for rule_id, static_rule_violation in static_rule_violations_by_rule_id.items():
            if static_rule_violation:
                print(f'Email {mail_row} is violating rule {rule_id} due to static analysis. Checking content analysis and topic analysis if needed')
            
                is_violated_rule_by_content_analysis = self._is_violating_rule(rule_id, content_analysis)
                is_violated_rule_by_topic_analysis = self._is_violating_rule(rule_id, topic_analysis)
            
                if is_violated_rule_by_content_analysis and is_violated_rule_by_topic_analysis:
                    print(f'Email {mail_row} is violating rule {rule_id} due to content analysis or topic analysis and therefore BLOCKED')
                    result.violated_rules.append(rule_id)
                else:
                    print(f'Email {mail_row} is not violating rule {rule_id} due to content analysis or topic analysis and therefore ALLOWED')
        
        return result
   
            
        

    def only_this_rule_violated(self, static_rule_violations_by_rule_id, rule_id):
        return static_rule_violations_by_rule_id[rule_id] == True and sum(static_rule_violations_by_rule_id.values())

    def analyze_static(self, processed_row) -> {str : bool}:
        static_rule_violations_by_rule_id: {str: bool} = {
            '1.1': self.static_analyzer.is_violating_rule_1_1(processed_row),
            '1.2': self.static_analyzer.is_violating_rule_1_2(processed_row),
            '1.3': self.static_analyzer.is_violating_rule_1_3(processed_row),
            '2.1': self.static_analyzer.is_violating_rule_2_1(processed_row),
            '2.2': self.static_analyzer.is_violating_rule_2_2(processed_row),
            '2.3': self.static_analyzer.is_violating_rule_2_3(processed_row)
        }
        
        return static_rule_violations_by_rule_id


    def _only_1_rules_are_violated(self, static_rule_violations_by_rule_id):
        rule_2_prefix = '2.'
        
        for rule_id, rule_violation in static_rule_violations_by_rule_id:
            if rule_id.contains(rule_2_prefix) and static_rule_violations_by_rule_id[rule_id]:
                return False
            
        return True

    def _is_violating_rule(self, rule_id, content_analysis: ContentAnalysisResult):
        if rule_id.contains('1.'):
            return False
        if rule_id is '2.1' or rule_id is '2.2':
            return content_analysis.is_sensitive
        elif rule_id is '2.3':
            return Enforcer._is_violating_rule_2_3(content_analysis)
        else:
            raise Exception(f'Rule {rule_id} is not supported')
        
    def _is_violating_rule(self, rule_id, topic_analysis: TopicAnalysisResult):
        if rule_id is '1.1':
            return topic_analysis.is_legal
        elif rule_id is '1.2':
            return topic_analysis.is_finance
        elif rule_id is '1.3':
            return topic_analysis.is_finance or topic_analysis.is_business
        elif rule_id is '2.1':
            return topic_analysis.is_finance
        elif rule_id is '2.2':
            return topic_analysis.is_business
        elif rule_id is '2.3':
            return False
        else: 
            raise Exception(f'Rule {rule_id} is not supported for topic analysis')
        


    @staticmethod
    def _is_violating_rule_2_3(content_analysis):
        return content_analysis.is_sensitive and (content_analysis.has_pii or content_analysis.quids_counter >= 2)


enforcer = Enforcer()

def classify_mail(mail_row) -> bool:
    enforcer_results = enforcer.enforce(mail_row)
    
    pass
    


IndentationError: expected an indented block after 'if' statement on line 41 (4084019371.py, line 44)